In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# matplotlib.use("Agg")

from ase.visualize import view

from ase.io import read
from agox.databases import Database
from agox.environments import Environment
from agox.utils.graph_sorting import Analysis

import glob
import numpy as np
from sklearn.decomposition import PCA
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

In [ ]:
## Set up the plotting environment
# matplotlib.rcParams.update(matplotlib.rcParamsDefault)
plt.rc('text', usetex=True)
plt.rc('font', family='cmr10', size=12)
plt.rcParams["axes.formatter.use_mathtext"] = True

In [ ]:
## Set the plotting parameters
seed = 0
identifier = ""
output_directory_prefix = "DOutput"

In [ ]:
## Set the descriptors
from agox.models.descriptors import SOAP
local_descriptor = local_descriptor = SOAP.from_species(["Al"], r_cut=5.0)

In [ ]:
## Set the calculators
from chgnet.model import CHGNetCalculator
from ase.calculators.singlepoint import SinglePointCalculator
calc = CHGNetCalculator()

In [ ]:
## Load known structures
poscar_files = glob.glob("../known_phases/*.vasp")
known_phases = []
opacity_list = []
known_phase_labels = []
for poscar_file in poscar_files:
  phase = read(poscar_file)
  opacity = None
  if "mp-134.vasp" in poscar_file:
    print("FCC")
    cell = phase.get_cell()
    cell = [ vec * 4.02 / np.linalg.norm(vec) for vec in cell]
    phase.set_cell(cell, scale_atoms=True)
    opacity = 1.0
    label = "FCC (mp-134)"
  elif "mp-2647008.vasp" in poscar_file:
    print("HCP")
    cell = phase.get_cell()
    cell = [ vec * 3.56 / np.linalg.norm(vec) for vec in cell]
    phase.set_cell(cell, scale_atoms=True)
    opacity = 0.6
    label = "HCP (mp-2647008)"
  elif "mp-998860.vasp" in poscar_file:
    print("BCC")
    cell = phase.get_cell()
    cell = [ vec * 3.10 / np.linalg.norm(vec) for vec in cell]
    phase.set_cell(cell, scale_atoms=True)
    opacity = 0.4
    label = "BCC (mp-998860)"
  elif "mp-1183144.vasp" in poscar_file:
    print("$\\alpha$")
    cell = phase.get_cell()
    # cell[0] = [ vec * 4.94 / np.linalg.norm(vec) for vec in cell]
    # cell[1] = [ vec * 4.94 / np.linalg.norm(vec) for vec in cell]
    cell[2] = cell[2] * 9.88 / np.linalg.norm(cell[2])
    phase.set_cell(cell, scale_atoms=True)
    opacity = 0.7
    label = "$\\alpha$ (mp-1183144)"
  else:
    print("Skipping ", poscar_file)
    continue
  opacity_list.append(opacity)
  known_phase_labels.append(label)
  phase.calc = calc
  known_phases.append(phase)

In [ ]:
## Load the unrelaxed structures
unrlxd_structures = read(output_directory_prefix+identifier+"/unrlxd_structures_seed"+str(seed)+".traj", index=":")
# for structure in unrlxd_structures:
#   structure.calc = calc

In [ ]:
## Load the relaxed structures
rlxd_structures = read(output_directory_prefix+identifier+"/rlxd_structures_seed"+str(seed)+".traj", index=":")
# for structure in rlxd_structures:
#   structure.calc = calc

In [ ]:
## This is now addressed by using SinglePointCalculator to store the energy in the Atoms object directly before writing to the trajectory file

# ## Read energies from energies_unrlxd_seed0.txt and add to the respective structures using a SinglePointCalculator
# ## The file has the form "index energy"
# ## This is done because there seem to be issues with storing the energy in the ASE trajectory file for some setups
# filename = output_directory_prefix+identifier+"/energies_unrlxd_seed"+str(seed)+".txt"
# with open(filename) as f:
#     for line in f:
#         index, energy = line.split()
#         index = int(index)
#         energy = float(energy)
#         unrlxd_structures[index].calc = SinglePointCalculator(unrlxd_structures[index], energy=energy * len(unrlxd_structures[index]))


# filename = output_directory_prefix+identifier+"/energies_rlxd_seed"+str(seed)+".txt"
# with open(filename) as f:
#     for line in f:
#         index, energy = line.split()
#         index = int(index)
#         energy = float(energy)
#         rlxd_structures[index].calc = SinglePointCalculator(rlxd_structures[index], energy=energy * len(rlxd_structures[index]))

In [ ]:
## Get the minimum energy
min_energy = np.min([structure.get_potential_energy()/len(structure) for structure in rlxd_structures])

In [ ]:
## Calculate energies per atom for each unrelaxed structure
energies_per_atom = [structure.get_potential_energy() / len(structure) for structure in unrlxd_structures]
unrlxd_delta_en_per_atom = np.array(energies_per_atom) - min_energy
print("Unrelaxed min energy: ", np.min(energies_per_atom))

In [ ]:
## Calculate energies per atom for each relaxed structure
energies_per_atom = [structure.get_potential_energy() / len(structure) for structure in rlxd_structures]
rlxd_delta_en_per_atom = np.array(energies_per_atom) - min_energy
print("Relaxed min energy: ", np.min(energies_per_atom))

In [ ]:
# print the min, max, mean, and std of the relaxed energies
print("Relaxed energies per atom: min = {:.2f}, max = {:.2f}, mean = {:.2f}, std = {:.2f}".format(
    np.min(rlxd_delta_en_per_atom), np.max(rlxd_delta_en_per_atom), 
    np.mean(rlxd_delta_en_per_atom), np.std(rlxd_delta_en_per_atom)))
# plot in latex table format
print("Aluminium search & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\".format(
    np.min(rlxd_delta_en_per_atom), np.max(rlxd_delta_en_per_atom), 
    np.mean(rlxd_delta_en_per_atom), np.std(rlxd_delta_en_per_atom)))

In [ ]:
## Set up the PCA
pca = PCA(n_components=2)

In [ ]:
## Get the 'super atom' descriptors for the unrelaxed structures
unrlxd_super_atoms = []
for structure in unrlxd_structures:
  unrlxd_super_atoms.append( np.mean(local_descriptor.get_features(structure), axis=0) )

In [ ]:
## Get the 'super atom' descriptors for the relaxed structures
rlxd_super_atoms = []
for structure in rlxd_structures:
  rlxd_super_atoms.append( np.mean(local_descriptor.get_features(structure), axis=0) )

In [ ]:
## Get the 'super atom' descriptors for the known structures
known_super_atoms = []
for structure in known_phases:
  known_super_atoms.append( np.mean(local_descriptor.get_features(structure), axis=0) )

In [ ]:
## Fit the PCA model to the unrelaxed or relaxed structures
rlxd_string = "rlxd"

In [ ]:
## Save pca model
import pickle
if True:
  pca.fit(np.squeeze([arr for arr in rlxd_super_atoms]))
  with open("pca_model_all_rlxd_"+str(seed)+".pkl", "wb") as f:
    pickle.dump(pca, f)

## Load pca model
with open("pca_model_all_"+rlxd_string+"_0.pkl", "rb") as f:
  pca = pickle.load(f)

In [ ]:
## Transform the unrelaxed and relaxed structures to the reduced space
unrlxd_X_reduced = pca.transform(np.squeeze([arr for arr in unrlxd_super_atoms]))
rlxd_X_reduced = pca.transform(np.squeeze([arr for arr in rlxd_super_atoms]))
known_X_reduced = pca.transform(np.squeeze([arr for arr in known_super_atoms]))

In [ ]:
## Get the index of the structure with the minimum energy
min_energy_index = np.argmin(rlxd_delta_en_per_atom)
print(min_energy_index)

In [ ]:
## Plot the PCA
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 6))

plt.subplots_adjust(wspace=0.05, hspace=0)

## Get the maximum energy for the colourbar
max_en = min(3.0, max(np.max(unrlxd_delta_en_per_atom), np.max(rlxd_delta_en_per_atom)))

## Plot the PCA
axes[0].scatter(unrlxd_X_reduced[:, 0], unrlxd_X_reduced[:, 1], c=unrlxd_delta_en_per_atom, cmap="viridis", vmin = 0, vmax = max_en)
axes[1].scatter(rlxd_X_reduced[:, 0], rlxd_X_reduced[:, 1], c=rlxd_delta_en_per_atom, cmap="viridis", vmin = 0, vmax = max_en)

# Sort pairs of opacity and X_reduced by opacity value (highest to lowest)
sorted_pairs = sorted(zip(opacity_list, known_X_reduced, known_phase_labels), key=lambda x: x[0], reverse=False)
## Add the minimum energy structures to the plot
for ax in axes:
    for i, X in enumerate(sorted_pairs):
        ax.scatter(X[1][0], X[1][1], s=200, edgecolor=[1.0, 1.0-X[0], 1.0-X[0], X[0]], facecolor='none', linewidth=2, label=X[2])
    if ax == axes[1]:
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles[::-1], labels[::-1], facecolor='white', framealpha=1.0, edgecolor='black', fancybox=False, bbox_to_anchor=(1.041, 1.0), fontsize=20, handletextpad=0.2, borderpad=0.3, handlelength=1)
  
## Add labels
fig.text(0.5, 0.0, 'Principal component 1', ha='center', fontsize=20)
axes[0].set_ylabel('Principal component 2', fontsize=20)
axes[0].set_title('Unrelaxed', fontsize=20)
axes[1].set_title('Relaxed', fontsize=20)
if rlxd_string == "rlxd":
    xlims = [-30, 70]
    ylims = [-5, 20]
else:
    xlims = [-42, 55]
    ylims = [-12, 30]

for ax in axes:
    ax.tick_params(axis='both', direction='in', length=6, labelsize=20)
    # ax.yaxis.set_major_locator(MultipleLocator(3))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))
    ax.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax.tick_params(axis='both', which='minor', length=3, direction='in')
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)

## Unify tick labels
xticks = axes[0].get_xticks()
xticks = xticks[(xticks >= xlims[0]) & (xticks <= xlims[1])]

axes[1].set_xticks(xticks)
axes[1].set_yticklabels([])
axes[0].tick_params(axis='x', labelbottom=True, top=True)
axes[1].tick_params(axis='x', labelbottom=True, top=True)
axes[0].tick_params(axis='y', labelbottom=True, right=True)
axes[1].tick_params(axis='y', labelbottom=True, right=True)

## Make axes[0] and axes[1] the same width
axes[0].set_box_aspect(1.7)
axes[1].set_box_aspect(1.7)

## Add colorbar next to the axes
cbar = fig.colorbar(axes[1].collections[0], ax=axes, orientation='vertical', fraction=0.085, pad=0.02)
cbar.ax.tick_params(labelsize=20)
cbar.ax.yaxis.set_major_locator(MultipleLocator(1))
cbar.ax.yaxis.set_minor_locator(AutoMinorLocator(2))
cbar.set_label('Formation energy (eV/atom)', fontsize=20)

## Save the figure
plt.savefig('Al_RAFFLE_pca_'+rlxd_string+'_fit_seed'+str(seed)+'.pdf', bbox_inches='tight', pad_inches=0, facecolor=fig.get_facecolor(), edgecolor='none')

In [ ]:
## Identify the line of structures in the lower right corner
for i in range(len(rlxd_X_reduced)):
  if rlxd_X_reduced[i, 0] > 40 and rlxd_X_reduced[i, 0] < 45 and rlxd_X_reduced[i, 1] > -5 and rlxd_X_reduced[i, 1] < 0:
    print(i)

In [ ]:
from raffle.generator import raffle_generator

generator = raffle_generator()
generator.distributions.read_gdfs( output_directory_prefix+identifier+"/gdfs_seed"+str(seed)+".txt" )
descriptor = generator.get_descriptor()

# Create a figure with 3 subplots side by side
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot for each n-body descriptor (2-body, 3-body, 4-body)
for j in range(3):
    # Calculate x-axis values
    x = np.arange(generator.distributions.cutoff_min[j],
                generator.distributions.cutoff_max[j] + generator.distributions.width[j],
                generator.distributions.width[j])

    # Plot on the respective subplot
    for idx in range(len(descriptor[j])):
        axes[j].plot(x, descriptor[j][idx,:],
                     color='black')

    # Set labels and title for each subplot
    # axes[j].set_ylabel('Descriptor value')
    # axes[j].set_title(f'{j+2}-body descriptor')

# set the legend font size
for ax in axes:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(16)

axes[0].set_ylabel('Distribution function (arb. units)', fontsize=20)
axes[0].set_xlabel('Bond length (Å)', fontsize=20)
axes[1].set_xlabel('3-body angle (radians)', fontsize=20)
axes[2].set_xlabel('4-body angle (radians)', fontsize=20)
axes[0].set_xlim(0, 6)
axes[1].set_xlim(0, np.pi)
axes[2].set_xlim(0, np.pi)

axes[0].set_ylim(0, None)
axes[1].set_ylim(0, 1.0)
axes[2].set_ylim(0, 1.0)

# reduce number of ticks to five at most
axes[0].xaxis.set_major_locator(plt.MaxNLocator(3))
axes[0].yaxis.set_major_locator(plt.MaxNLocator(3))

# set x ticks to 0, 1, 2, 3
axes[1].set_xticks([0, 1, 2, 3])
axes[2].set_xticks([0, 1, 2, 3])
# axes[1].set_yticks([0, 0.1, 0.2])
# axes[2].set_yticks([0, 0.05, 0.1])

# have the ticks point intwards and on both sides
for ax in axes:
    ax.tick_params(axis='both', which='major', direction='in', length=10, width=1)
    ax.tick_params(axis='both', which='minor', direction='in', length=5, width=1)
    ax.tick_params(axis='x', which='both', bottom=True, top=True)
    ax.tick_params(axis='y', which='both', left=True, right=True)

axes[1].yaxis.set_major_locator(MultipleLocator(0.5))
axes[2].yaxis.set_major_locator(MultipleLocator(0.5))
# add minor ticks to all axes
for ax in axes:
    ax.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))
    


# axes[0].set_xlabel('Distance (Å)')
# axes[1].set_xlabel('3-body angle (radians)')
# axes[2].set_xlabel('Improper dihedral angle (radians)')
# plt.tight_layout()
# plt.show()

# get figures closer together
plt.subplots_adjust(wspace=0.1, hspace=0)
# remove labels for y axis for the second and third plots

plt.savefig('Al_RAFFLE'+identifier+'_gen_descriptor_rlxd_seed'+str(seed)+'.pdf', bbox_inches='tight', pad_inches=0, facecolor=fig.get_facecolor(), edgecolor='none')
